## Dangjin

In [1]:
base_date = '20210706' # API base date
date1 = '2021-07-07'    # Dataframe date 1 (today)
d1 = '20210707'
date2 = '2021-07-08'    # Dataframe date 2 tomorrow
d2 = '20210708'
filedate = '0707'      # file name date (today)

In [2]:
import pandas as pd
import urllib
import urllib.request
import json
from pandas.io.json import json_normalize

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst' #동네예보조회
queryParams = '?' + urllib.parse.urlencode({urllib.parse.quote_plus('ServiceKey') : 'GGG0Gj970rHlfVO7vBhr/frNJ71SbhPWjgLlEqFs6PlXX5L07/knfolyspZAB7MP6bIBbYS6pHWDr85m+KwLEw==',
                              urllib.parse.quote_plus('numOfRows') : '200',  
                              urllib.parse.quote_plus('dataType') : 'JSON', 
                              urllib.parse.quote_plus('base_date') : base_date, #발표일자
                              urllib.parse.quote_plus('base_time') : '1400', #발표시각 
                              urllib.parse.quote_plus('nx') : '53', # ulsan 102 , dangjin'53'
                              urllib.parse.quote_plus('ny') : '114',}) #ulsan 83 dangjin '114'

response = urllib.request.urlopen(url + queryParams).read()
response = json.loads(response)

In [3]:
response

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
  'body': {'dataType': 'JSON',
   'items': {'item': [{'baseDate': '20210706',
      'baseTime': '1400',
      'category': 'POP',
      'fcstDate': '20210706',
      'fcstTime': '1800',
      'fcstValue': '30',
      'nx': 53,
      'ny': 114},
     {'baseDate': '20210706',
      'baseTime': '1400',
      'category': 'PTY',
      'fcstDate': '20210706',
      'fcstTime': '1800',
      'fcstValue': '0',
      'nx': 53,
      'ny': 114},
     {'baseDate': '20210706',
      'baseTime': '1400',
      'category': 'R06',
      'fcstDate': '20210706',
      'fcstTime': '1800',
      'fcstValue': '0',
      'nx': 53,
      'ny': 114},
     {'baseDate': '20210706',
      'baseTime': '1400',
      'category': 'REH',
      'fcstDate': '20210706',
      'fcstTime': '1800',
      'fcstValue': '75',
      'nx': 53,
      'ny': 114},
     {'baseDate': '20210706',
      'baseTime': '1400',
      'category': 'S06',
      'fcstD

In [4]:
for i, data in enumerate(response['response']['body']['items']['item']):
    print (i, data)

0 {'baseDate': '20210706', 'baseTime': '1400', 'category': 'POP', 'fcstDate': '20210706', 'fcstTime': '1800', 'fcstValue': '30', 'nx': 53, 'ny': 114}
1 {'baseDate': '20210706', 'baseTime': '1400', 'category': 'PTY', 'fcstDate': '20210706', 'fcstTime': '1800', 'fcstValue': '0', 'nx': 53, 'ny': 114}
2 {'baseDate': '20210706', 'baseTime': '1400', 'category': 'R06', 'fcstDate': '20210706', 'fcstTime': '1800', 'fcstValue': '0', 'nx': 53, 'ny': 114}
3 {'baseDate': '20210706', 'baseTime': '1400', 'category': 'REH', 'fcstDate': '20210706', 'fcstTime': '1800', 'fcstValue': '75', 'nx': 53, 'ny': 114}
4 {'baseDate': '20210706', 'baseTime': '1400', 'category': 'S06', 'fcstDate': '20210706', 'fcstTime': '1800', 'fcstValue': '0', 'nx': 53, 'ny': 114}
5 {'baseDate': '20210706', 'baseTime': '1400', 'category': 'SKY', 'fcstDate': '20210706', 'fcstTime': '1800', 'fcstValue': '4', 'nx': 53, 'ny': 114}
6 {'baseDate': '20210706', 'baseTime': '1400', 'category': 'T3H', 'fcstDate': '20210706', 'fcstTime': '1

In [5]:
fcst_df = pd.DataFrame()

fcst_df['Forecast_time'] = [f'{dates} {hour}:00' for dates in [date1, date2] for hour in range(0,24)]
row_idx = 0

for i, data in enumerate(response['response']['body']['items']['item']):
    #if i < 62: # 여기 예보시간에 따라 변경해야 함!
    if data['fcstDate'] == d1 or data['fcstDate'] == d2:
        if data['category']=='REH':
            fcst_df.loc[row_idx, 'Humidity'] = float(data['fcstValue'])
            print('category:Humidity', '/fcst_date: ', data['fcstDate'],'/fcstTime:', data['fcstTime'], '/category: ', data['category'], '/baseTime:', base_date, data['baseTime'],  '/fcstValue:', data['fcstValue'])
        elif data['category']=='T3H':
            fcst_df.loc[row_idx, 'Temperature'] = float(data['fcstValue'])
            print('category:Temperature,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='SKY':
            fcst_df.loc[row_idx, 'Cloud'] = float(data['fcstValue'])
            print('category:Cloud,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='VEC':
            fcst_df.loc[row_idx, 'WindDirection'] = float(data['fcstValue'])
            print('category:WindDirection,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='PTY':
            fcst_df.loc[row_idx, 'PrecipitationForm'] = float(data['fcstValue'])
        elif data['category']=='POP':
            fcst_df.loc[row_idx, 'PrecipitationProb'] = float(data['fcstValue'])
        elif data['category']=='WSD':
            fcst_df.loc[row_idx, 'WindSpeed'] = float(data['fcstValue'])
            print('category:WindSpeed,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'], '\n')
            row_idx+=3

category:Humidity /fcst_date:  20210707 /fcstTime: 0000 /category:  REH /baseTime: 20210706 1400 /fcstValue: 90
category:Cloud, SKY baseTime: 1400 , fcstTime: 0000 , fcstValue: 4
category:Temperature, T3H baseTime: 1400 , fcstTime: 0000 , fcstValue: 23
category:WindDirection, VEC baseTime: 1400 , fcstTime: 0000 , fcstValue: 216
category:WindSpeed, WSD baseTime: 1400 , fcstTime: 0000 , fcstValue: 1.4 

category:Humidity /fcst_date:  20210707 /fcstTime: 0300 /category:  REH /baseTime: 20210706 1400 /fcstValue: 95
category:Cloud, SKY baseTime: 1400 , fcstTime: 0300 , fcstValue: 4
category:Temperature, T3H baseTime: 1400 , fcstTime: 0300 , fcstValue: 22
category:WindDirection, VEC baseTime: 1400 , fcstTime: 0300 , fcstValue: 222
category:WindSpeed, WSD baseTime: 1400 , fcstTime: 0300 , fcstValue: 1.5 

category:Humidity /fcst_date:  20210707 /fcstTime: 0600 /category:  REH /baseTime: 20210706 1400 /fcstValue: 95
category:Cloud, SKY baseTime: 1400 , fcstTime: 0600 , fcstValue: 4
category:Te

In [6]:
fcst_df

Forecast_time  PrecipitationProb  PrecipitationForm  Humidity  Cloud  \
0    2021-07-07 0:00               30.0                0.0      90.0    4.0   
1    2021-07-07 1:00                NaN                NaN       NaN    NaN   
2    2021-07-07 2:00                NaN                NaN       NaN    NaN   
3    2021-07-07 3:00               30.0                0.0      95.0    4.0   
4    2021-07-07 4:00                NaN                NaN       NaN    NaN   
5    2021-07-07 5:00                NaN                NaN       NaN    NaN   
6    2021-07-07 6:00               30.0                0.0      95.0    4.0   
7    2021-07-07 7:00                NaN                NaN       NaN    NaN   
8    2021-07-07 8:00                NaN                NaN       NaN    NaN   
9    2021-07-07 9:00               30.0                0.0      85.0    4.0   
10  2021-07-07 10:00                NaN                NaN       NaN    NaN   
11  2021-07-07 11:00                NaN                NaN       NaN    NaN   
12  2021-07-07 12:00               60.0                1.0      80.0    4.0   
13  2021-07-07 13:00                NaN                NaN       NaN    NaN   
14  2021-07-07 14:00                NaN                NaN       NaN    NaN   
15  2021-07-07 15:00               60.0                1.0      90.0    4.0   
16  2021-07-07 16:00                NaN                NaN       NaN    NaN   
17  2021-07-07 17:00                NaN                NaN       NaN    NaN   
18  2021-07-07 18:00               60.0                1.0      90.0    4.0   
19  2021-07-07 19:00                NaN                NaN       NaN    NaN   
20  2021-07-07 20:00                NaN                NaN       NaN    NaN   
21  2021-07-07 21:00               80.0                1.0      95.0    4.0   
22  2021-07-07 22:00                NaN                NaN       NaN    NaN   
23  2021-07-07 23:00                NaN                NaN       NaN    NaN   
24   2021-07-08 0:00               80.0                1.0      95.0    4.0   
25   2021-07-08 1:00                NaN                NaN       NaN    NaN   
26   2021-07-08 2:00                NaN                NaN       NaN    NaN   
27   2021-07-08 3:00               60.0                1.0      95.0    4.0   
28   2021-07-08 4:00                NaN                NaN       NaN    NaN   
29   2021-07-08 5:00                NaN                NaN       NaN    NaN   
30   2021-07-08 6:00               20.0                0.0      95.0    3.0   
31   2021-07-08 7:00                NaN                NaN       NaN    NaN   
32   2021-07-08 8:00                NaN                NaN       NaN    NaN   
33   2021-07-08 9:00               20.0                0.0      90.0    3.0   
34  2021-07-08 10:00                NaN                NaN       NaN    NaN   
35  2021-07-08 11:00                NaN                NaN       NaN    NaN   
36  2021-07-08 12:00               20.0                0.0      85.0    3.0   
37  2021-07-08 13:00                NaN                NaN       NaN    NaN   
38  2021-07-08 14:00                NaN                NaN       NaN    NaN   
39  2021-07-08 15:00                0.0                0.0      80.0    1.0   
40  2021-07-08 16:00                NaN                NaN       NaN    NaN   
41  2021-07-08 17:00                NaN                NaN       NaN    NaN   
42  2021-07-08 18:00                0.0                0.0      85.0    1.0   
43  2021-07-08 19:00                NaN                NaN       NaN    NaN   
44  2021-07-08 20:00                NaN                NaN       NaN    NaN   
45  2021-07-08 21:00                0.0                0.0      95.0    1.0   
46  2021-07-08 22:00                NaN                NaN       NaN    NaN   
47  2021-07-08 23:00                NaN                NaN       NaN    NaN   

    Temperature  WindDirection  WindSpeed  
0          23.0          216.0        1.4  
1           NaN            NaN        NaN  
2  

In [7]:
fcst_df = fcst_df.interpolate()
fcst_df = fcst_df.iloc[:48]

In [8]:
fcst_df

Forecast_time  PrecipitationProb  PrecipitationForm   Humidity  \
0    2021-07-07 0:00          30.000000           0.000000  90.000000   
1    2021-07-07 1:00          30.000000           0.000000  91.666667   
2    2021-07-07 2:00          30.000000           0.000000  93.333333   
3    2021-07-07 3:00          30.000000           0.000000  95.000000   
4    2021-07-07 4:00          30.000000           0.000000  95.000000   
5    2021-07-07 5:00          30.000000           0.000000  95.000000   
6    2021-07-07 6:00          30.000000           0.000000  95.000000   
7    2021-07-07 7:00          30.000000           0.000000  91.666667   
8    2021-07-07 8:00          30.000000           0.000000  88.333333   
9    2021-07-07 9:00          30.000000           0.000000  85.000000   
10  2021-07-07 10:00          40.000000           0.333333  83.333333   
11  2021-07-07 11:00          50.000000           0.666667  81.666667   
12  2021-07-07 12:00          60.000000           1.000000  80.000000   
13  2021-07-07 13:00          60.000000           1.000000  83.333333   
14  2021-07-07 14:00          60.000000           1.000000  86.666667   
15  2021-07-07 15:00          60.000000           1.000000  90.000000   
16  2021-07-07 16:00          60.000000           1.000000  90.000000   
17  2021-07-07 17:00          60.000000           1.000000  90.000000   
18  2021-07-07 18:00          60.000000           1.000000  90.000000   
19  2021-07-07 19:00          66.666667           1.000000  91.666667   
20  2021-07-07 20:00          73.333333           1.000000  93.333333   
21  2021-07-07 21:00          80.000000           1.000000  95.000000   
22  2021-07-07 22:00          80.000000           1.000000  95.000000   
23  2021-07-07 23:00          80.000000           1.000000  95.000000   
24   2021-07-08 0:00          80.000000           1.000000  95.000000   
25   2021-07-08 1:00          73.333333           1.000000  95.000000   
26   2021-07-08 2:00          66.666667           1.000000  95.000000   
27   2021-07-08 3:00          60.000000           1.000000  95.000000   
28   2021-07-08 4:00          46.666667           0.666667  95.000000   
29   2021-07-08 5:00          33.333333           0.333333  95.000000   
30   2021-07-08 6:00          20.000000           0.000000  95.000000   
31   2021-07-08 7:00          20.000000           0.000000  93.333333   
32   2021-07-08 8:00          20.000000           0.000000  91.666667   
33   2021-07-08 9:00          20.000000           0.000000  90.000000   
34  2021-07-08 10:00          20.000000           0.000000  88.333333   
35  2021-07-08 11:00          20.000000           0.000000  86.666667   
36  2021-07-08 12:00          20.000000           0.000000  85.000000   
37  2021-07-08 13:00          13.333333           0.000000  83.333333   
38  2021-07-08 14:00           6.666667           0.000000  81.666667   
39  2021-07-08 15:00           0.000000           0.000000  80.000000   
40  2021-07-08 16:00           0.000000           0.000000  81.666667   
41  2021-07-08 17:00           0.000000           0.000000  83.333333   
42  2021-07-08 18:00           0.000000           0.000000  85.000000   
43  2021-07-08 19:00           0.000000           0.000000  88.333333   
44  2021-07-08 20:00           0.000000           0.000000  91.666667   
45  2021-07-08 21:00           0.000000           0.000000  95.000000   
46  2021-07-08 22:00           0.000000           0.000000  95.000000   
47  2021-07-08 23:00           0.000000           0.000000  95.000000   

       Cloud  Temperature  WindDirection  WindSpeed  
0   4.000000    23.000000     216.000000   1.400000  
1   4.000000    22.666667     218.000000   1.433333  
2   4.000000    22.333333     220.000000   1.466667  
3   4.000000    22.000000     222.000000   1.500000  
4   4.000000    22.000000     189.666667   1.533333  
5   4.000000    22.000000     157.333333   1.566667  
6   4.000000    22.000000     125.000000   1.600000

In [9]:
fcst_df.to_csv("./data/dangjin_" + filedate + ".csv", index = False)

## Ulsan

In [10]:
import pandas as pd
import urllib
import urllib.request
import json
from pandas.io.json import json_normalize

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst' #동네예보조회
queryParams = '?' + urllib.parse.urlencode({urllib.parse.quote_plus('ServiceKey') : 'GGG0Gj970rHlfVO7vBhr/frNJ71SbhPWjgLlEqFs6PlXX5L07/knfolyspZAB7MP6bIBbYS6pHWDr85m+KwLEw==',
                              urllib.parse.quote_plus('numOfRows') : '200',  
                              urllib.parse.quote_plus('dataType') : 'JSON', 
                              urllib.parse.quote_plus('base_date') : base_date, #발표일자
                              urllib.parse.quote_plus('base_time') : '1400', #발표시각 (오후2시)
                              urllib.parse.quote_plus('nx') : '102', # ulsan 102 , dangjin'53'
                              urllib.parse.quote_plus('ny') : '83',}) #ulsan 83 dangjin '114'

response = urllib.request.urlopen(url + queryParams).read()
response = json.loads(response)

In [11]:
fcst_df = pd.DataFrame()

fcst_df['Forecast_time'] = [f'{dates} {hour}:00' for dates in [date1, date2] for hour in range(0,24)]
row_idx = 0

for i, data in enumerate(response['response']['body']['items']['item']):
    #if i < 62: # 여기 예보시간에 따라 변경해야 함!
    if data['fcstDate'] == d1 or data['fcstDate'] == d2:
        if data['category']=='REH':
            fcst_df.loc[row_idx, 'Humidity'] = float(data['fcstValue'])
            print('category:Humidity', '/fcst_date: ', data['fcstDate'],'/fcstTime:', data['fcstTime'], '/category: ', data['category'], '/baseTime:', base_date, data['baseTime'],  '/fcstValue:', data['fcstValue'])
        elif data['category']=='T3H':
            fcst_df.loc[row_idx, 'Temperature'] = float(data['fcstValue'])
            print('category:Temperature,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='SKY':
            fcst_df.loc[row_idx, 'Cloud'] = float(data['fcstValue'])
            print('category:Cloud,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='VEC':
            fcst_df.loc[row_idx, 'WindDirection'] = float(data['fcstValue'])
            print('category:WindDirection,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='PTY':
            fcst_df.loc[row_idx, 'PrecipitationForm'] = float(data['fcstValue'])
        elif data['category']=='POP':
            fcst_df.loc[row_idx, 'PrecipitationProb'] = float(data['fcstValue'])
        elif data['category']=='WSD':
            fcst_df.loc[row_idx, 'WindSpeed'] = float(data['fcstValue'])
            print('category:WindSpeed,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'], '\n')
            row_idx+=3

category:Humidity /fcst_date:  20210707 /fcstTime: 0000 /category:  REH /baseTime: 20210706 1400 /fcstValue: 90
category:Cloud, SKY baseTime: 1400 , fcstTime: 0000 , fcstValue: 4
category:Temperature, T3H baseTime: 1400 , fcstTime: 0000 , fcstValue: 26
category:WindDirection, VEC baseTime: 1400 , fcstTime: 0000 , fcstValue: 255
category:WindSpeed, WSD baseTime: 1400 , fcstTime: 0000 , fcstValue: 2.4 

category:Humidity /fcst_date:  20210707 /fcstTime: 0300 /category:  REH /baseTime: 20210706 1400 /fcstValue: 85
category:Cloud, SKY baseTime: 1400 , fcstTime: 0300 , fcstValue: 4
category:Temperature, T3H baseTime: 1400 , fcstTime: 0300 , fcstValue: 26
category:WindDirection, VEC baseTime: 1400 , fcstTime: 0300 , fcstValue: 261
category:WindSpeed, WSD baseTime: 1400 , fcstTime: 0300 , fcstValue: 2 

category:Humidity /fcst_date:  20210707 /fcstTime: 0600 /category:  REH /baseTime: 20210706 1400 /fcstValue: 90
category:Cloud, SKY baseTime: 1400 , fcstTime: 0600 , fcstValue: 4
category:Temp

In [12]:
fcst_df #오늘, 내일

Forecast_time  PrecipitationProb  PrecipitationForm  Humidity  Cloud  \
0    2021-07-07 0:00               61.0                1.0      90.0    4.0   
1    2021-07-07 1:00                NaN                NaN       NaN    NaN   
2    2021-07-07 2:00                NaN                NaN       NaN    NaN   
3    2021-07-07 3:00               60.0                1.0      85.0    4.0   
4    2021-07-07 4:00                NaN                NaN       NaN    NaN   
5    2021-07-07 5:00                NaN                NaN       NaN    NaN   
6    2021-07-07 6:00               60.0                1.0      90.0    4.0   
7    2021-07-07 7:00                NaN                NaN       NaN    NaN   
8    2021-07-07 8:00                NaN                NaN       NaN    NaN   
9    2021-07-07 9:00               60.0                1.0      85.0    4.0   
10  2021-07-07 10:00                NaN                NaN       NaN    NaN   
11  2021-07-07 11:00                NaN                NaN       NaN    NaN   
12  2021-07-07 12:00               60.0                1.0      80.0    4.0   
13  2021-07-07 13:00                NaN                NaN       NaN    NaN   
14  2021-07-07 14:00                NaN                NaN       NaN    NaN   
15  2021-07-07 15:00               60.0                1.0      80.0    4.0   
16  2021-07-07 16:00                NaN                NaN       NaN    NaN   
17  2021-07-07 17:00                NaN                NaN       NaN    NaN   
18  2021-07-07 18:00               60.0                1.0      85.0    4.0   
19  2021-07-07 19:00                NaN                NaN       NaN    NaN   
20  2021-07-07 20:00                NaN                NaN       NaN    NaN   
21  2021-07-07 21:00               60.0                1.0      90.0    4.0   
22  2021-07-07 22:00                NaN                NaN       NaN    NaN   
23  2021-07-07 23:00                NaN                NaN       NaN    NaN   
24   2021-07-08 0:00               30.0                0.0      85.0    4.0   
25   2021-07-08 1:00                NaN                NaN       NaN    NaN   
26   2021-07-08 2:00                NaN                NaN       NaN    NaN   
27   2021-07-08 3:00               30.0                0.0      90.0    4.0   
28   2021-07-08 4:00                NaN                NaN       NaN    NaN   
29   2021-07-08 5:00                NaN                NaN       NaN    NaN   
30   2021-07-08 6:00               30.0                0.0      90.0    4.0   
31   2021-07-08 7:00                NaN                NaN       NaN    NaN   
32   2021-07-08 8:00                NaN                NaN       NaN    NaN   
33   2021-07-08 9:00               30.0                0.0      85.0    4.0   
34  2021-07-08 10:00                NaN                NaN       NaN    NaN   
35  2021-07-08 11:00                NaN                NaN       NaN    NaN   
36  2021-07-08 12:00               30.0                0.0      80.0    4.0   
37  2021-07-08 13:00                NaN                NaN       NaN    NaN   
38  2021-07-08 14:00                NaN                NaN       NaN    NaN   
39  2021-07-08 15:00               60.0                1.0      80.0    4.0   
40  2021-07-08 16:00                NaN                NaN       NaN    NaN   
41  2021-07-08 17:00                NaN                NaN       NaN    NaN   
42  2021-07-08 18:00               30.0                0.0      80.0    4.0   
43  2021-07-08 19:00                NaN                NaN       NaN    NaN   
44  2021-07-08 20:00                NaN                NaN       NaN    NaN   
45  2021-07-08 21:00               30.0                0.0      90.0    4.0   
46  2021-07-08 22:00                NaN                NaN       NaN    NaN   
47  2021-07-08 23:00                NaN                NaN       NaN    NaN   

    Temperature  WindDirection  WindSpeed  
0          26.0          255.0        2.4  
1           NaN            NaN        NaN  
2  

In [13]:
fcst_df = fcst_df.interpolate()
fcst_df = fcst_df.iloc[:48]

In [14]:
fcst_df

Forecast_time  PrecipitationProb  PrecipitationForm   Humidity  Cloud  \
0    2021-07-07 0:00          61.000000           1.000000  90.000000    4.0   
1    2021-07-07 1:00          60.666667           1.000000  88.333333    4.0   
2    2021-07-07 2:00          60.333333           1.000000  86.666667    4.0   
3    2021-07-07 3:00          60.000000           1.000000  85.000000    4.0   
4    2021-07-07 4:00          60.000000           1.000000  86.666667    4.0   
5    2021-07-07 5:00          60.000000           1.000000  88.333333    4.0   
6    2021-07-07 6:00          60.000000           1.000000  90.000000    4.0   
7    2021-07-07 7:00          60.000000           1.000000  88.333333    4.0   
8    2021-07-07 8:00          60.000000           1.000000  86.666667    4.0   
9    2021-07-07 9:00          60.000000           1.000000  85.000000    4.0   
10  2021-07-07 10:00          60.000000           1.000000  83.333333    4.0   
11  2021-07-07 11:00          60.000000           1.000000  81.666667    4.0   
12  2021-07-07 12:00          60.000000           1.000000  80.000000    4.0   
13  2021-07-07 13:00          60.000000           1.000000  80.000000    4.0   
14  2021-07-07 14:00          60.000000           1.000000  80.000000    4.0   
15  2021-07-07 15:00          60.000000           1.000000  80.000000    4.0   
16  2021-07-07 16:00          60.000000           1.000000  81.666667    4.0   
17  2021-07-07 17:00          60.000000           1.000000  83.333333    4.0   
18  2021-07-07 18:00          60.000000           1.000000  85.000000    4.0   
19  2021-07-07 19:00          60.000000           1.000000  86.666667    4.0   
20  2021-07-07 20:00          60.000000           1.000000  88.333333    4.0   
21  2021-07-07 21:00          60.000000           1.000000  90.000000    4.0   
22  2021-07-07 22:00          50.000000           0.666667  88.333333    4.0   
23  2021-07-07 23:00          40.000000           0.333333  86.666667    4.0   
24   2021-07-08 0:00          30.000000           0.000000  85.000000    4.0   
25   2021-07-08 1:00          30.000000           0.000000  86.666667    4.0   
26   2021-07-08 2:00          30.000000           0.000000  88.333333    4.0   
27   2021-07-08 3:00          30.000000           0.000000  90.000000    4.0   
28   2021-07-08 4:00          30.000000           0.000000  90.000000    4.0   
29   2021-07-08 5:00          30.000000           0.000000  90.000000    4.0   
30   2021-07-08 6:00          30.000000           0.000000  90.000000    4.0   
31   2021-07-08 7:00          30.000000           0.000000  88.333333    4.0   
32   2021-07-08 8:00          30.000000           0.000000  86.666667    4.0   
33   2021-07-08 9:00          30.000000           0.000000  85.000000    4.0   
34  2021-07-08 10:00          30.000000           0.000000  83.333333    4.0   
35  2021-07-08 11:00          30.000000           0.000000  81.666667    4.0   
36  2021-07-08 12:00          30.000000           0.000000  80.000000    4.0   
37  2021-07-08 13:00          40.000000           0.333333  80.000000    4.0   
38  2021-07-08 14:00          50.000000           0.666667  80.000000    4.0   
39  2021-07-08 15:00          60.000000           1.000000  80.000000    4.0   
40  2021-07-08 16:00          50.000000           0.666667  80.000000    4.0   
41  2021-07-08 17:00          40.000000           0.333333  80.000000    4.0   
42  2021-07-08 18:00          30.000000           0.000000  80.000000    4.0   
43  2021-07-08 19:00          30.000000           0.000000  83.333333    4.0   
44  2021-07-08 20:00          30.000000           0.000000  86.666667    4.0   
45  2021-07-08 21:00          30.000000           0.000000  90.000000    4.0   
46  2021-07-08 22:00          30.000000           0.000000  90.000000    4.0   
47  2021-07-08 23:00          30.000000           0.000000  90.000000    4.0   

    Temperature  WindDirection  WindSpeed  
0     26.000000     255.000000   2.400000 

In [15]:
fcst_df.to_csv("./data/ulsan_" + filedate + ".csv", index = False)